In [1]:
import pandas as pd
import numpy as np

import os

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [2]:
if 'label' in train.columns:
    train.drop(columns=['label'], inplace=True)
train_total = pd.merge(train, train_labels, on='id')

In [3]:
labels = train_labels['label'].drop_duplicates()

In [7]:
print(len(train_total))
print(len(train_labels)*600)

1875000
1875000


In [8]:
aug_cnt = 700

for idx, label in labels.iteritems():
    if os.path.exists(f'./aug/train_labels_{label}.csv'):
        print(f'./aug/train_labels_{label}.csv', ' is already exists!')
    else :
        tmps = train_total[train_total['label'] == label].copy()
        while len(train_labels[train_labels['label'] == label]) < aug_cnt:
            id_list = tmps['id'].drop_duplicates()
            print(f'{label}:', len(train_labels[train_labels['label'] == label]))
            for _, _id in id_list.iteritems(): 
                if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                    tmp = tmps[tmps['id'] == _id].copy()
                    tmp['acc_x'] = tmp['acc_x'] + tmp['acc_x'].std() * np.random.normal(0.0, 1.0, 600)
                    tmp['acc_y'] = tmp['acc_y'] + tmp['acc_y'].std() * np.random.normal(0.0, 1.0, 600)
                    tmp['acc_z'] = tmp['acc_z'] + tmp['acc_z'].std() * np.random.normal(0.0, 1.0, 600)
                    tmp['gy_x'] = tmp['gy_x'] + tmp['gy_x'].std() * np.random.normal(0.0, 1.0, 600)
                    tmp['gy_y'] = tmp['gy_y'] + tmp['gy_y'].std() * np.random.normal(0.0, 1.0, 600)
                    tmp['gy_z'] = tmp['gy_z'] + tmp['gy_z'].std() * np.random.normal(0.0, 1.0, 600)
                    newid = train_labels['id'].max() + 1
                    tmp['id'] = newid
                    tmps = pd.concat([tmps, tmp])
                    train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

                if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                    for rate in [0.1*i for i in range(5, 15)]:
                        tmp = tmps[tmps['id'] == _id].copy()
                        tmp['acc_x'] = tmp['acc_x']*rate
                        tmp['acc_y'] = tmp['acc_y']*rate
                        tmp['acc_z'] = tmp['acc_z']*rate
                        tmp['gy_x'] = tmp['gy_x']*rate
                        tmp['gy_y'] = tmp['gy_y']*rate
                        tmp['gy_z'] = tmp['gy_z']*rate
                        newid = train_labels['id'].max() + 1
                        tmp['id'] = newid
                        tmps = pd.concat([tmps, tmp])
                        train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

                if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                    tmp = tmps[tmps['id'] == _id].copy()
                    tmp.sort_values(by='time', ascending=False)
                    newid = train_labels['id'].max() + 1
                    tmp['id'] = newid
                    tmps = pd.concat([tmps, tmp])
                    train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)
    
        print(label)
        tmps.to_csv(f'./aug/train_features_{label}.csv')
        train_labels[train_labels['label'] == label].to_csv(f'./aug/train_labels_{label}.csv')

37: 35
37: 455
37
26
3: 23
3: 299
3
39: 20
39: 260
39
28: 55
28
6: 24
6: 312
6
60: 48
60: 624
60
35: 30
35: 390
35
34: 22
34: 286
34
30: 35
30: 455
30
59: 23
59: 299
59
15: 25
15: 325
15
40: 34
40: 442
40
50: 37
50: 481
50
48: 25
48: 325
48
8: 97
8
54: 23
54: 299
54
16: 22
16: 286
16
4: 35
4: 455
4
58: 20
58: 260
58
21: 27
21: 351
21
18: 47
18: 611
18
51: 24
51: 312
51
25: 24
25: 312
25
24: 35
24: 455
24
17: 27
17: 351
17
49: 30
49: 390
49
47: 26
47: 338
47
27: 34
27: 442
27
36: 28
36: 364
36
2: 20
2: 260
2
38: 20
38: 260
38
14: 25
14: 325
14
10: 20
10: 260
10
55: 37
55: 481
55
57: 20
57: 260
57
0: 12
0: 156
0
1: 21
1: 273
1
46: 20
46: 260
46
43: 35
43: 455
43
52: 12
52: 156
52
53: 13
53: 169
53
5: 25
5: 325
5
56: 36
56: 468
56
23: 20
23: 260
23
44: 21
44: 273
44
42: 20
42: 260
42
19: 20
19: 260
19
32: 18
32: 234
32
45: 22
45: 286
45
29: 20
29: 260
29
31: 20
31: 260
31
11: 23
11: 299
11
7: 26
7: 338
7
41: 20
41: 260
41
12: 12
12: 156
12
9: 37
9: 481
9
20: 26
20: 338
20
13: 12
13: 156
1

In [9]:
import pandas as pd
import numpy as np

train_labels=pd.read_csv('train_labels.csv')
labels = train_labels['label'].drop_duplicates()

train_feature = pd.DataFrame()
train_label = pd.DataFrame()

i = 0
for _, label in labels.iteritems():
    tmp_feature = pd.read_csv(f"aug/train_features_{label}.csv")
    tmp_feature = tmp_feature.iloc[:int(len(tmp_feature)/600)*600,:]
    train_feature = pd.concat([train_feature, tmp_feature])

    tmp_label = pd.read_csv(f"aug/train_labels_{label}.csv").iloc[:int(len(tmp_feature)/600),:]
    train_label = pd.concat([train_label, tmp_label])
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [10]:
train_feature.to_csv('train_feature_aug.csv')
train_label.to_csv('train_label_aug.csv')

In [1]:
import pandas as pd
import numpy as np

train_feature = pd.read_csv('sampled_feature_2.csv')
train_label = pd.read_csv('sampled_label_2.csv')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D

In [4]:
def sampling_func(data, sample_cnt):
    np.random.seed(123)
    N = len(data)
    sample_n = sample_cnt #int(len(data)*sample_pct) # integer
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample


sampled_label = train_label.groupby(by='label').apply(sampling_func, sample_cnt=400)

In [5]:
query = f"id in {list(sampled_label['id'].to_numpy())}"
sampled_feature = train_feature.query(query)

In [6]:
sampled_feature.to_csv('sampled_feature_2.csv')
sampled_label.to_csv('sampled_label_2.csv')

In [3]:
y=tf.keras.utils.to_categorical(train_label['label']) 
print(y.shape)

(24400, 61)


In [4]:
X=tf.reshape(np.array(train_feature.iloc[:,3:9]),[-1, 600, 6])
print(X.shape)

(24400, 600, 6)


In [5]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, num_filters=32, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.num_filters = num_filters
        self.activation = tf.keras.activations.get(activation)

    def _default_Conv1D(self, filters, kernel_size):
        return tf.keras.layers.Conv1D(filters=filters,
                                      kernel_size=kernel_size,
                                      strides=1,
                                      padding='same',
                                      activation='relu',
                                      use_bias=False)

    def call(self, inputs):
        
        z_bottleneck = self._default_Conv1D(filters=self.num_filters, kernel_size=1)(inputs)
        z_maxpool = tf.keras.layers.MaxPool1D(pool_size=3, strides=1, padding='same')(inputs)

        z1 = self._default_Conv1D(filters=self.num_filters, kernel_size=10)(z_bottleneck)
        z2 = self._default_Conv1D(filters=self.num_filters, kernel_size=20)(z_bottleneck)
        z3 = self._default_Conv1D(filters=self.num_filters, kernel_size=40)(z_bottleneck)
        z4 = self._default_Conv1D(filters=self.num_filters, kernel_size=1)(z_maxpool)

        z = tf.keras.layers.Concatenate(axis=2)([z1, z2, z3, z4])
        z = tf.keras.layers.BatchNormalization()(z)

        return self.activation(z)


In [6]:
def shortcut_layer(inputs, z_inception):
    z_shortcut = tf.keras.layers.Conv1D(filters=int(z_inception.shape[-1]), 
                                        kernel_size=1, 
                                        padding='same', 
                                        use_bias=False)(inputs)

    z_shortcut = tf.keras.layers.BatchNormalization()(z_shortcut)

    z = tf.keras.layers.Add()([z_shortcut, z_inception])

    return tf.keras.layers.Activation('relu')(z)


def build_model(input_shape, num_classes, num_modules=6):
    input_layer = tf.keras.layers.Input(input_shape)
    z = input_layer
    z_residual = input_layer

    for i in range(num_modules):
        z = InceptionModule()(z)
        if i%3 == 2:
            z = shortcut_layer(z_residual, z)
            z_residual = z

    gap_layer = tf.keras.layers.GlobalAveragePooling1D()(z)
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(gap_layer)

    model = tf.keras.models.Model(inputs= input_layer, outputs= output_layer)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    return model

In [7]:
model = build_model((600, 6), 61, num_modules=6)

In [11]:
model.fit(X,y, validation_split=0.2, epochs=100, batch_size=128)